In [ ]:
# code to add data to an excel file



In [ ]:
# !pip install openpyxl

In [4]:
# pip install openpyxl pandas
from typing import Iterable, Optional
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string

In [ ]:



def write_value(path: str, sheet_name: str, cell_addr: str, value) -> None:
    """
    Write a single value to an EXACT cell in an EXISTING sheet.
    Will NOT create sheets.

    Parameters
    ----------
    path : str
        Path to the .xlsx file (modified in place).
    sheet_name : str
        Existing sheet name.
    cell_addr : str
        A1-style address (e.g., "B4").
    value : any
        The value to write. Strings starting with '=' are written as formulas.
    """
    wb = load_workbook(path)
    if sheet_name not in wb.sheetnames:
        raise KeyError(f"Sheet '{sheet_name}' not found in workbook.")
    ws = wb[sheet_name]
    ws[cell_addr] = value
    wb.save(path)


def write_dataframe_columns(
    path: str,
    sheet_name: str,
    start_cell: str,
    df: pd.DataFrame,
    columns: Optional[Iterable[str]] = None,
    include_header: bool = True,
    include_index: bool = False,
) -> None:
    """
    Write selected DataFrame columns to an EXACT start cell in an EXISTING sheet.
    Preserves the rest of the sheet. Will NOT create sheets.

    Parameters
    ----------
    path : str
        Path to the .xlsx file (modified in place).
    sheet_name : str
        Existing sheet name.
    start_cell : str
        Top-left A1 address where the table should be written (e.g., "D10").
    df : pd.DataFrame
        Source DataFrame.
    columns : iterable[str] or None
        Subset of columns to write. If None, writes all df columns.
    include_header : bool
        If True, writes column names in the first row.
    include_index : bool
        If True, writes the DataFrame index as the first column.
    """
    wb = load_workbook(path)
    if sheet_name not in wb.sheetnames:
        raise KeyError(f"Sheet '{sheet_name}' not found in workbook.")
    ws = wb[sheet_name]

    # subset columns if requested
    data = df if columns is None else df.loc[:, list(columns)]

    # figure out start row/col from A1 address
    col_letters, row_idx = coordinate_from_string(start_cell)
    start_col = column_index_from_string(col_letters)
    start_row = row_idx

    # build a 2D list of values to write
    rows_to_write = []

    # header
    if include_header:
        header_row = []
        if include_index:
            header_row.append(df.index.name if df.index.name is not None else "")
        header_row.extend(list(data.columns))
        rows_to_write.append(header_row)

    # body
    for idx, (_, row) in enumerate(data.iterrows(), start=0):
        out_row = []
        if include_index:
            out_row.append(data.index[idx])
        # replace NaN with None so cells are blank
        out_row.extend([None if pd.isna(v) else v for v in row.to_list()])
        rows_to_write.append(out_row)

    # write to worksheet
    r = start_row
    for row_vals in rows_to_write:
        c = start_col
        for val in row_vals:
            ws.cell(row=r, column=c, value=val)
            c += 1
        r += 1

    wb.save(path)


In [ ]:
# 1) Write a single value
write_value(
    path="Model_A.xlsx",
    sheet_name="Inputs",
    cell_addr="B4",
    value=123.45,  # could also be "=SUM(C2:C10)" if you want a formula
)

# 2) Write selected DataFrame columns starting at a specific cell
df = pd.DataFrame({
    "scenario": ["Base", "High", "Low"],
    "capex": [1_000_000, 1_200_000, 900_000],
    "opex":  [50_000, 55_000, 48_000],
    "notes": ["ok", "stress", None],
})

write_dataframe_columns(
    path="Model_A.xlsx",
    sheet_name="Results",
    start_cell="D10",          # top-left corner for the table
    df=df,
    columns=["scenario", "capex", "opex"],  # only these columns
    include_header=True,
    include_index=False,
)